# Data Analysis





### Import libraries

In [16]:
import pandas as pd


### Read data

In [18]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d mlg-ulb/creditcardfraud


The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.



  0%|          | 0.00/66.0M [00:00<?, ?B/s]
  2%|▏         | 1.00M/66.0M [00:00<00:18, 3.75MB/s]
  6%|▌         | 4.00M/66.0M [00:00<00:05, 11.4MB/s]
  9%|▉         | 6.00M/66.0M [00:00<00:07, 8.60MB/s]
 12%|█▏        | 8.00M/66.0M [00:01<00:07, 8.19MB/s]
 14%|█▎        | 9.00M/66.0M [00:01<00:07, 8.23MB/s]
 15%|█▌        | 10.0M/66.0M [00:01<00:07, 7.54MB/s]
 17%|█▋        | 11.0M/66.0M [00:01<00:09, 6.33MB/s]
 20%|█▉        | 13.0M/66.0M [00:01<00:06, 8.26MB/s]
 21%|██        | 14.0M/66.0M [00:01<00:06, 8.50MB/s]
 24%|██▍       | 16.0M/66.0M [00:02<00:05, 9.11MB/s]
 26%|██▌       | 17.0M/66.0M [00:02<00:10, 4.81MB/s]
 29%|██▉       | 19.0M/66.0M [00:02<00:08, 6.00MB/s]
 30%|███       | 20.0M/66.0M [00:03<00:08, 5.72MB/s]
 32%|███▏      | 21.0M/66.0M [00:03<00:08, 5.83MB/s]
 33%|███▎      | 22.0M/66.0M [00:03<00:07, 6.22MB/s]
 35%|███▍      | 23.0M/66.0M [00:03<00:06, 6.74MB/s]
 36%|███▋      | 24.0M/66.0M [00:03<00:06, 7.16MB/s]
 39%|███▉      | 26.0M/66.0M [00:03<00:04, 9.91MB/s]
 

In [20]:
! unzip creditcardfraud.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
credidcard_fraud_df = pd.read_csv('creditcard.csv')


### General stats about the data

* Attributes
* df.describe()


In [4]:
credidcard_fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [5]:
# Take aways: None null values (All data segments are complete)
# All features are type float64, accept for the Class which is int64

In [21]:
# cheack for dublicates

credidcard_fraud_df.drop_duplicates()
credidcard_fraud_df.info()

# No duplicates

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
#### Fit data


In [22]:
from scipy import stats
import matplotlib.pyplot as plt
# Stolen Code :o Then modified

list_of_dists = ['alpha','anglit','arcsine','beta','betaprime','bradford',
                 'burr','burr12','cauchy','chi','chi2','cosine','dgamma',
                 'dweibull','erlang','expon','exponnorm','exponweib','exponpow',
                 'f','fatiguelife','fisk','foldcauchy','foldnorm','genlogistic',
                 'genpareto','gennorm','genexpon','genextreme','gausshyper',
                 'gamma','gengamma','genhalflogistic','gilbrat','gompertz',
                 'gumbel_r','gumbel_l','halfcauchy','halflogistic','halfnorm',
                 'halfgennorm','hypsecant','invgamma','invgauss','invweibull',
                 'johnsonsb','johnsonsu','kstwobign','laplace','levy','levy_l',
                 'logistic','loggamma','loglaplace','lognorm','lomax','maxwell',
                 'mielke','nakagami','ncx2','ncf','nct','norm','pareto',
                 'pearson3','powerlaw','powerlognorm','powernorm','rdist',
                 'reciprocal','rayleigh','rice','recipinvgauss','semicircular',
                 't','triang','truncexpon','truncnorm','tukeylambda','uniform',
                 'vonmises','vonmises_line','wald','weibull_min','weibull_max']


def find_a_fitting_dist(data):
    '''Finds a fitting distribution for data'''
    results = list()
    for i in list_of_dists:
        dist = getattr(stats, i)
        param = dist.fit(data)
        a = stats.kstest(data, i, args=param)
        results.append((i,a[0],a[1]))
  
    results.sort(key=lambda x:float(x[2]), reverse=True)

    # return (<name of dist>, args*)
    return results[0][0], getattr(stats, results[0][0]).fit(data)


def plot_data_w_dist(data, dist, title = "Title", n_bins=40):
    plt.title(title, fontsize=22, fontweight="bold")
    x= np.linspace(min(data), max(data))
    plt.hist(data, bins = n_bins, density=True)
    eval('plt.plot(x, stats.{}.pdf(x, {}))'.format(str(dist[0]), 
        ', '.join(list([str(x) for x in dist[1]]))))



Matplotlib is building the font cache; this may take a moment.


In [ ]:
import warnings
# ignore warnings
warnings.filterwarnings('ignore')
# TODO: Remove ? or disable

# For each feature plot the data with the fitting curve

for i , f_id in enumerate(credidcard_fraud_df.columns ):
    if i>1: pass
    # ToDo add p-value to title
    plot_data_w_dist(credidcard_fraud_df[f_id], find_a_fitting_dist(credidcard_fraud_df[f_id]), f_id, n_bins=100)


### Clean and wrangle

### Do some useful stuff